<a href="https://colab.research.google.com/github/KasraOstad/Vineyard-Classification-in-Sussex/blob/master/GEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook was used to extract data from all of the relevant satellite images for each polygon in our study area

In [ ]:
!pip install folium
!pip install geehydro

import ee
from datetime import datetime as dt
from IPython.display import Image
import pandas as pd
import matplotlib.pyplot as plt
import time
import folium
import geehydro

#Authenticate to the Earth Engine servers
ee.Authenticate()

#Initialize the API
ee.Initialize()

  Created wheel for geehydro: filename=geehydro-0.2.0-py2.py3-none-any.whl size=10113 sha256=3390ca5acef281827592ea9305b4bba281000b2ef071321546ebb138b8d90872
  Stored in directory: /root/.cache/pip/wheels/c5/07/67/5fa6e7271b46bbe0acafdc7105bbee27a39ab7132d251d822d
Successfully built geehydro
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=b1POq4pXuy2aS5ohvkAvoVYTji6-RslrO0DCzSLv1vI&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter

In [ ]:
#Load Satellite of interest
collection=('COPERNICUS/S2_SR')
s2=ee.ImageCollection(collection)

#Define Area of interest
area_list= list ([(-0.831,51.1),(-0.831,50.711),(0.127,50.711),(0.127,51.1),(-0.831,51.1)])
AOI=ee.Geometry.Polygon(area_list)

#Filter image collection with AOI
s2=s2.filterBounds(AOI)

#Filter image collection with date
s2=s2.filterDate('2019-07-01','2020-01-01')

#Total number of images after filtering 
print('Total number:', s2.size().getInfo())

#Filter image collection with cloud coverage meta
least_cloudy_s2 =  s2.filterMetadata('CLOUD_COVERAGE_ASSESSMENT','less_than',10)
print('Total number:', least_cloudy_s2.size().getInfo())

zero_cloud_list=least_cloudy_s2.toList(least_cloudy_s2.size())

#Find dates of our filtered image collection
dates=[]
for i in range(0, zero_cloud_list.size().getInfo()):
  image=ee.Image(zero_cloud_list.get(i))
  date=ee.Number.parse(image.date().format("YYYYMMdd"))
  dates.append(date.getInfo())

print(dates)  


Total number: 150
Total number: 18
[20190723, 20190723, 20190820, 20190822, 20190825, 20190825, 20190827, 20190904, 20190914, 20190914, 20190919, 20190921, 20190921, 20191118, 20191203, 20191225, 20191225, 20191230]


In [ ]:
#Create Mosaic from images for our ROI and date
aug_25_19=[ee.Image(zero_cloud_list.get(4)),ee.Image(zero_cloud_list.get(5))]
im_25_08_19= ee.ImageCollection(aug_25_19)
mosaic = im_25_08_19.mosaic()


#Define NDVI index from mosaic and visualise the region and image
ndvi = mosaic.normalizedDifference(['B8', 'B4'])

palette = ['red', 'yellow', 'green']
ndvi_parameters = {'min': 0,
                   'max': 1,
                   'dimensions': 512,
                   'palette': palette,
                   'region': AOI}
                   

sussex_map=folium.Map(location=[50.9665,-0.2275],zoom_start=7)
sussex_map.addLayer(ndvi, ndvi_parameters)
sussex_map

In [ ]:
#Compute Other Vegetation Indicies
ndwi=mosaic.normalizedDifference(['B8','B11'])

evi2 = mosaic.expression(
    '2.5 * ((NIR - RED) / (NIR + 2.4 * RED + 1))', {
    'NIR': mosaic.select('B8'),
    'RED': mosaic.select('B4')}
)

cl=mosaic.expression(
    '(RED3 / RED1) -1',{
    'RED3': mosaic.select('B7'),
    'RED1': mosaic.select('B5')
    }
)

#Create gg.image with all vegetation bands
veg=ndvi.addBands(ndwi)
veg=veg.addBands(evi2)
veg=veg.addBands(cl)

#Rename Vegetation bands
veg=veg.select(veg.bandNames().getInfo(),['NDVI', 'NDWI', 'EVI2', 'CL'])

#Filter out redundant S2_SR bands
s2=mosaic.select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12','QA60'])

#Add vegetation indicies to sentinel 2 image
s2_veg=s2.addBands(veg)

print(s2_veg.bandNames().getInfo())

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'QA60', 'NDVI', 'NDWI', 'EVI2', 'CL']


In [ ]:
def s1_find_date (start,end):
  sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD')\
              .filterDate(start,end)
  sentinel1=sentinel1.filterBounds(AOI)

  # Filter by metadata properties.
  # 1. Filter to get images with VV and VH dual polarization.
  # 2. Filter to get images collected in interferometric wide swath mode.
  #IW, with VV+VH polarisation over land

  vh = sentinel1.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
                .filter(ee.Filter.eq('instrumentMode', 'IW')) 

  vh_list=vh.toList(vh.size())

  dates=[]

  for i in range(0, vh_list.size().getInfo()):
    image=ee.Image(vh_list.get(i))
    date=ee.Number.parse(image.date().format("YYYYMMdd"))
    dates.append(date.getInfo())
  
  print(dates)
  return vh_list



In [ ]:
#Display Sentinel 1 image
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
  
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

s1_im_list=s1_find_date('2019-08-20','2019-09-01')
s1_im= ee.Image(s1_im_list.get(2))

tk = s1_im.getMapId({'min': [-25, -20, -25], 'max': [0, 10, 0]})
center=[51.1,-0.831]
Mapdisplay(center,{'composite':tk},zoom_start=7)



[20190820, 20190822, 20190825, 20190827, 20190827, 20190829, 20190830, 20190821, 20190824, 20190824, 20190826, 20190828, 20190831, 20190831]


In [ ]:
s1_im_list=s1_find_date('2019-04-20','2019-05-01')
s1_im2= ee.Image(s1_im_list.get(13))

tk = s1_im2.getMapId({'min': [-25, -20, -25], 'max': [0, 10, 0]})
Mapdisplay(center,{'composite':tk},zoom_start=7)

[20190420, 20190422, 20190424, 20190427, 20190429, 20190429, 20190421, 20190421, 20190423, 20190425, 20190426, 20190426, 20190428, 20190430]


In [ ]:
s1_im_list=s1_find_date('2019-06-22','2019-06-24')

s1_im3= ee.Image(s1_im_list.get(0))
tk = s1_im3.getMapId({'min': [-25, -20, -25], 'max': [0, 10, 0]})
Mapdisplay(center,{'composite':tk},zoom_start=7)

[20190623, 20190622]


In [ ]:
#Extract satellite band information for all our polygons
#Restricted to only 3 bands at a time
def feature_extract(fc,img,fol):
  asset_name='users/aghajaniwork/'+fc
  polygons= ee.FeatureCollection(asset_name)

  polygon_stats=img.reduceRegions(**{
    'collection':polygons,
    'reducer':ee.Reducer.median(),
    'scale':  10 
  })

  task=ee.batch.Export.table.toDrive(**{
  'collection': polygon_stats,
  'description': fc,
  'fileFormat': 'SHP',
  'folder':fol
  })

  task.start()

  while task.active():
    print('Extraction Processing...'.format(task.id))
    time.sleep(30)

  return  task.status() 


In [ ]:
feature_extract('2polygons_all',s1_im,'s1_extraction')

Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...


{'creation_timestamp_ms': 1595365052057,
 'description': '2polygons_all',
 'destination_uris': ['https://drive.google.com/#folders/1Bll2j2-yPPiX0D8crovKkgAbF2Oi02LX',
  'https://drive.google.com/#folders/1Bll2j2-yPPiX0D8crovKkgAbF2Oi02LX',
  'https://drive.google.com/#folders/1Bll2j2-yPPiX0D8crovKkgAbF2Oi02LX',
  'https://drive.google.com/#folders/1Bll2j2-yPPiX0D8crovKkgAbF2Oi02LX',
  'https://drive.google.com/#folders/1Bll2j2-yPPiX0D8crovKkgAbF2Oi02LX',
  'https://drive.google.com/#folders/1Bll2j2-yPPiX0D8crovKkgAbF2Oi02LX'],
 'id': 'UOFZ4KAGLD5FWRKJSMPBEBGJ',
 'name': 'projects/earthengine-legacy/operations/UOFZ4KAGLD5FWRKJSMPBEBGJ',
 'start_timestamp_ms': 1595365081798,
 'state': 'COMPLETED',
 'task_type': 'EXPORT_FEATURES',
 'update_timestamp_ms': 1595366801478}

In [ ]:
feature_extract('2polygons_all',s1_im2,'s1_extraction2')

Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...


{'creation_timestamp_ms': 1597096723101,
 'description': '2polygons_all',
 'destination_uris': ['https://drive.google.com/#folders/1PuAGn7Z-Q509QwPVjUn3cemrpHSDO15y',
  'https://drive.google.com/#folders/1PuAGn7Z-Q509QwPVjUn3cemrpHSDO15y',
  'https://drive.google.com/#folders/1PuAGn7Z-Q509QwPVjUn3cemrpHSDO15y',
  'https://drive.google.com/#folders/1PuAGn7Z-Q509QwPVjUn3cemrpHSDO15y',
  'https://drive.google.com/#folders/1PuAGn7Z-Q509QwPVjUn3cemrpHSDO15y',
  'https://drive.google.com/#folders/1PuAGn7Z-Q509QwPVjUn3cemrpHSDO15y'],
 'id': 'UTEIHZ3QVHJ2I3WXFXPTNTXO',
 'name': 'projects/earthengine-legacy/operations/UTEIHZ3QVHJ2I3WXFXPTNTXO',
 'start_timestamp_ms': 1597096733822,
 'state': 'COMPLETED',
 'task_type': 'EXPORT_FEATURES',
 'update_timestamp_ms': 1597098337205}

In [ ]:
feature_extract('2polygons_all',s1_im3,'s1_extraction4')

Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...


{'creation_timestamp_ms': 1597189933704,
 'description': '2polygons_all',
 'destination_uris': ['https://drive.google.com/#folders/129idSZoT40Yxt0PqTT7YunHmEVYABxhB',
  'https://drive.google.com/#folders/129idSZoT40Yxt0PqTT7YunHmEVYABxhB',
  'https://drive.google.com/#folders/129idSZoT40Yxt0PqTT7YunHmEVYABxhB',
  'https://drive.google.com/#folders/129idSZoT40Yxt0PqTT7YunHmEVYABxhB',
  'https://drive.google.com/#folders/129idSZoT40Yxt0PqTT7YunHmEVYABxhB',
  'https://drive.google.com/#folders/129idSZoT40Yxt0PqTT7YunHmEVYABxhB'],
 'id': '2CU7BB2YYQ7SUWMM37WDBNBF',
 'name': 'projects/earthengine-legacy/operations/2CU7BB2YYQ7SUWMM37WDBNBF',
 'start_timestamp_ms': 1597189946507,
 'state': 'COMPLETED',
 'task_type': 'EXPORT_FEATURES',
 'update_timestamp_ms': 1597191485787}

In [ ]:
veg_1=veg.select(['NDVI', 'NDWI'])

feature_extract('2polygons_all',veg_1,'veg_index1')


Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...


{'creation_timestamp_ms': 1595421730419,
 'description': '2polygons_all',
 'destination_uris': ['https://drive.google.com/#folders/1ccmXbeHvGnpPhfpIqIqec1Bcsf8ljGoW',
  'https://drive.google.com/#folders/1ccmXbeHvGnpPhfpIqIqec1Bcsf8ljGoW',
  'https://drive.google.com/#folders/1ccmXbeHvGnpPhfpIqIqec1Bcsf8ljGoW',
  'https://drive.google.com/#folders/1ccmXbeHvGnpPhfpIqIqec1Bcsf8ljGoW',
  'https://drive.google.com/#folders/1ccmXbeHvGnpPhfpIqIqec1Bcsf8ljGoW',
  'https://drive.google.com/#folders/1ccmXbeHvGnpPhfpIqIqec1Bcsf8ljGoW'],
 'id': '2E55I3WW6DQRQCYNT3LVWTHR',
 'name': 'projects/earthengine-legacy/operations/2E55I3WW6DQRQCYNT3LVWTHR',
 'start_timestamp_ms': 1595421861230,
 'state': 'COMPLETED',
 'task_type': 'EXPORT_FEATURES',
 'update_timestamp_ms': 1595422724585}

In [ ]:
veg_2=veg.select(['EVI2', 'CL'])

feature_extract('2polygons_all',veg_2,'veg_index2')


Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...


{'creation_timestamp_ms': 1595423051996,
 'description': '2polygons_all',
 'destination_uris': ['https://drive.google.com/#folders/1zHIkGCAkRhywFG4fUePHAii_7xsbuj-t',
  'https://drive.google.com/#folders/1zHIkGCAkRhywFG4fUePHAii_7xsbuj-t',
  'https://drive.google.com/#folders/1zHIkGCAkRhywFG4fUePHAii_7xsbuj-t',
  'https://drive.google.com/#folders/1zHIkGCAkRhywFG4fUePHAii_7xsbuj-t',
  'https://drive.google.com/#folders/1zHIkGCAkRhywFG4fUePHAii_7xsbuj-t',
  'https://drive.google.com/#folders/1zHIkGCAkRhywFG4fUePHAii_7xsbuj-t'],
 'id': 'M2PH5LDDX2P77BTPHL264NJ2',
 'name': 'projects/earthengine-legacy/operations/M2PH5LDDX2P77BTPHL264NJ2',
 'start_timestamp_ms': 1595423141250,
 'state': 'COMPLETED',
 'task_type': 'EXPORT_FEATURES',
 'update_timestamp_ms': 1595423831430}

In [ ]:
s2_batch1=s2.select(['B1', 'B2'])
feature_extract('2polygons_all',s2_batch1,'s2_batch1')
#  'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12','QA60'])


Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...


{'creation_timestamp_ms': 1595426871440,
 'description': '2polygons_all',
 'destination_uris': ['https://drive.google.com/#folders/1FZGsvcscYbQBSGxQ32NKAxBdH9U3l5_4',
  'https://drive.google.com/#folders/1FZGsvcscYbQBSGxQ32NKAxBdH9U3l5_4',
  'https://drive.google.com/#folders/1FZGsvcscYbQBSGxQ32NKAxBdH9U3l5_4',
  'https://drive.google.com/#folders/1FZGsvcscYbQBSGxQ32NKAxBdH9U3l5_4',
  'https://drive.google.com/#folders/1FZGsvcscYbQBSGxQ32NKAxBdH9U3l5_4',
  'https://drive.google.com/#folders/1FZGsvcscYbQBSGxQ32NKAxBdH9U3l5_4'],
 'id': 'BHMGHB4OINZUOCVZCYN5NA7B',
 'name': 'projects/earthengine-legacy/operations/BHMGHB4OINZUOCVZCYN5NA7B',
 'start_timestamp_ms': 1595426879195,
 'state': 'COMPLETED',
 'task_type': 'EXPORT_FEATURES',
 'update_timestamp_ms': 1595427626701}

In [ ]:
s2_batch2=s2.select(['B3', 'B4'])
feature_extract('2polygons_all',s2_batch2,'s2_batch2')

Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...


{'creation_timestamp_ms': 1595427657480,
 'description': '2polygons_all',
 'destination_uris': ['https://drive.google.com/#folders/1K9IuTl6Acr9NrkISzLiRWjP1cB8gXBce',
  'https://drive.google.com/#folders/1K9IuTl6Acr9NrkISzLiRWjP1cB8gXBce',
  'https://drive.google.com/#folders/1K9IuTl6Acr9NrkISzLiRWjP1cB8gXBce',
  'https://drive.google.com/#folders/1K9IuTl6Acr9NrkISzLiRWjP1cB8gXBce',
  'https://drive.google.com/#folders/1K9IuTl6Acr9NrkISzLiRWjP1cB8gXBce',
  'https://drive.google.com/#folders/1K9IuTl6Acr9NrkISzLiRWjP1cB8gXBce'],
 'id': 'H7HF5LC5XRHXKOYE4V4ERYCA',
 'name': 'projects/earthengine-legacy/operations/H7HF5LC5XRHXKOYE4V4ERYCA',
 'start_timestamp_ms': 1595427678879,
 'state': 'COMPLETED',
 'task_type': 'EXPORT_FEATURES',
 'update_timestamp_ms': 1595428519541}

In [ ]:
s2_batch3=s2.select(['B5', 'B6'])
feature_extract('2polygons_all',s2_batch3,'s2_batch3')

Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...


{'creation_timestamp_ms': 1595428533364,
 'description': '2polygons_all',
 'destination_uris': ['https://drive.google.com/#folders/1ZZEStcB6I0iKpR298Y6_BO51tSZC9U-b',
  'https://drive.google.com/#folders/1ZZEStcB6I0iKpR298Y6_BO51tSZC9U-b',
  'https://drive.google.com/#folders/1ZZEStcB6I0iKpR298Y6_BO51tSZC9U-b',
  'https://drive.google.com/#folders/1ZZEStcB6I0iKpR298Y6_BO51tSZC9U-b',
  'https://drive.google.com/#folders/1ZZEStcB6I0iKpR298Y6_BO51tSZC9U-b',
  'https://drive.google.com/#folders/1ZZEStcB6I0iKpR298Y6_BO51tSZC9U-b'],
 'id': 'ICTQ3VVRVLXKPIJPJCXMTTAD',
 'name': 'projects/earthengine-legacy/operations/ICTQ3VVRVLXKPIJPJCXMTTAD',
 'start_timestamp_ms': 1595428554220,
 'state': 'COMPLETED',
 'task_type': 'EXPORT_FEATURES',
 'update_timestamp_ms': 1595429708867}

In [ ]:
s2_batch4=s2.select(['B7', 'B8'])
feature_extract('2polygons_all',s2_batch4,'s2_batch4')

Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...


{'creation_timestamp_ms': 1595429710681,
 'description': '2polygons_all',
 'destination_uris': ['https://drive.google.com/#folders/1YQ0PifJRBamKOtL5yo9ll24OHYoMLKhX',
  'https://drive.google.com/#folders/1YQ0PifJRBamKOtL5yo9ll24OHYoMLKhX',
  'https://drive.google.com/#folders/1YQ0PifJRBamKOtL5yo9ll24OHYoMLKhX',
  'https://drive.google.com/#folders/1YQ0PifJRBamKOtL5yo9ll24OHYoMLKhX',
  'https://drive.google.com/#folders/1YQ0PifJRBamKOtL5yo9ll24OHYoMLKhX',
  'https://drive.google.com/#folders/1YQ0PifJRBamKOtL5yo9ll24OHYoMLKhX'],
 'id': 'TSZYNVYBRAJX2JSBHDNVHAFJ',
 'name': 'projects/earthengine-legacy/operations/TSZYNVYBRAJX2JSBHDNVHAFJ',
 'start_timestamp_ms': 1595429722699,
 'state': 'COMPLETED',
 'task_type': 'EXPORT_FEATURES',
 'update_timestamp_ms': 1595431109846}

In [ ]:
s2_batch5=s2.select(['B8A', 'B9'])
feature_extract('2polygons_all',s2_batch5,'s2_batch5')

Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...


{'creation_timestamp_ms': 1595431129681,
 'description': '2polygons_all',
 'destination_uris': ['https://drive.google.com/#folders/18lWi7Pi28222Ktt4E_vATZ-YJJEdId8T',
  'https://drive.google.com/#folders/18lWi7Pi28222Ktt4E_vATZ-YJJEdId8T',
  'https://drive.google.com/#folders/18lWi7Pi28222Ktt4E_vATZ-YJJEdId8T',
  'https://drive.google.com/#folders/18lWi7Pi28222Ktt4E_vATZ-YJJEdId8T',
  'https://drive.google.com/#folders/18lWi7Pi28222Ktt4E_vATZ-YJJEdId8T',
  'https://drive.google.com/#folders/18lWi7Pi28222Ktt4E_vATZ-YJJEdId8T'],
 'id': '773QE4L7NZILO76X3AOPCQDI',
 'name': 'projects/earthengine-legacy/operations/773QE4L7NZILO76X3AOPCQDI',
 'start_timestamp_ms': 1595431154493,
 'state': 'COMPLETED',
 'task_type': 'EXPORT_FEATURES',
 'update_timestamp_ms': 1595431953161}

In [ ]:
s2_batch6=s2.select(['B11', 'B12','QA60'])
feature_extract('2polygons_all',s2_batch6,'s2_batch6')

Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...
Extraction Processing...


{'creation_timestamp_ms': 1595431975969,
 'description': '2polygons_all',
 'destination_uris': ['https://drive.google.com/#folders/1jg5Z4dHnJ80E5YrOdIyfEnih6LMP99zl',
  'https://drive.google.com/#folders/1jg5Z4dHnJ80E5YrOdIyfEnih6LMP99zl',
  'https://drive.google.com/#folders/1jg5Z4dHnJ80E5YrOdIyfEnih6LMP99zl',
  'https://drive.google.com/#folders/1jg5Z4dHnJ80E5YrOdIyfEnih6LMP99zl',
  'https://drive.google.com/#folders/1jg5Z4dHnJ80E5YrOdIyfEnih6LMP99zl',
  'https://drive.google.com/#folders/1jg5Z4dHnJ80E5YrOdIyfEnih6LMP99zl'],
 'id': 'PZE7QGKV6VDQMY3E6IQQDAVL',
 'name': 'projects/earthengine-legacy/operations/PZE7QGKV6VDQMY3E6IQQDAVL',
 'start_timestamp_ms': 1595432013795,
 'state': 'COMPLETED',
 'task_type': 'EXPORT_FEATURES',
 'update_timestamp_ms': 1595432864219}